In [ ]:
# # Get data from SOLR and save it to a file
# from urllib.request import urlopen
# import json
# connection = urlopen('https://shapenet.org/solr/models3d/select?q=datasets%3AShapeNetCore+AND+wnhypersynsets%3A02828884&rows=100000&fl=fullId%2Ccategory%2Cwnsynset%2Cwnlemmas%2Cup%2Cfront%2Cunit%2Caligned.dims&wt=json&indent=true')
# response = json.load(connection)
# print(response['response']['numFound'], "documents found.")
# print(response)
# Print the name of each document.

# for doc in response['response']['docs']:
#   print(doc)

# with open('data.json', 'w') as outfile:
#     json.dump(response, outfile)


In [ ]:
from tqdm import tqdm#
from pyvista import examples
import os
import numpy as np
import pyvista as pv
from math import dist
import numpy as np
# pv.set_jupyter_backend('pythreejs')

DATA_ROOT = "data\\shapenet_samples"

def outRayFinder(pts):
    if len(pts) == 0:
        return None, None
    # Find the first element in pts that is differnt from pts[0] and return that point onelin
    if len(pts) > 2:
        points = np.array(pts)
        idx = np.where(np.isclose(points, points[0])  != [True, True, True])[0][1]
        return pts[0], pts[idx]
    else:
        return pts[0], pts[1]


class Model:
    def __init__(self, model_path):
        self.model_path = model_path
        self.obj_path = model_path + "\\model.obj"
        self.mtl_path = model_path + "\\model.mtl"
        # Append all textures to a list within image folder
        self.textures = []
        # Error handle if no images folder
        try:
            for file in os.listdir(model_path + "\\images"):
                if file.endswith(".jpg"):
                    self.textures.append(model_path + "\\images\\" + file)
                elif file.endswith(".png"):
                    self.textures.append(model_path + "\\images\\" + file)
        except:
            pass

    def get_model_path(self):
        return self.model_path
    
    def get_obj_path(self):
        return self.obj_path

    def get_mtl_path(self):
        return self.mtl_path

    def get_textures(self):
        return self.textures

    def get_texture(self, index):
        if len(self.textures) == 0:
            return None
        else:
            return self.textures[index]



class ModelLoader:
    def __init__(self, data_root):
        self.data_root = data_root
        # Map model paths from catagory to model path
        self.model_paths = {}

        for catagory in os.listdir(data_root):
            catagory_path = os.path.join(data_root, catagory)
            model_p = []
            for model in os.listdir(catagory_path):
                model_path = os.path.join(catagory_path, model)
                model_p.append(Model(model_path))
            self.model_paths[catagory] = model_p

    def get_model_paths(self):
        return self.model_paths

    def get_model_paths_by_catagory(self, catagory):
        return self.model_paths[catagory]

    def get_model_paths_by_catagory_and_model(self, catagory, model):
        return self.model_paths[catagory][model]


def createBaseMesh(mesh):#
    # Add a thin box below the mesh
    bounds = mesh.bounds
    rnge = (bounds[1] - bounds[0], bounds[5] - bounds[4], bounds[3] - bounds[2]) # x z y
    print(rnge)

    expand = 10
    height = rnge[2] * 0.05
    center = np.array(mesh.center)
    center -= [0, mesh.center[2] + (bounds[3] + (height/3 )) ,0  ]

    width = rnge[0] * (1 + expand)
    length = rnge[1] * (1 + expand)
    base_mesh = pv.Cube(center, width, height, length)
    # base_mesh.rotate_x(-90, inplace=True)
    return base_mesh


data = ModelLoader(DATA_ROOT)


In [ ]:
pv.global_theme.background = (135, 206, 235)
pv.global_theme.smooth_shading = True
pv.global_theme.anti_aliasing = 'fxaa'


rotation = 180
catagory = "cow_mesh"
id_ = 0
image_res = 256
windowSize = 1000
show_edges = False
show_pts = False
show_planes = False
backend = "pythreejs"
#  Sort Data Paths
tex_path = data.get_model_paths()[catagory][id_].get_texture(0)
if tex_path is not None:
    obj_texture = pv.read_texture(tex_path)
obj_mesh = pv.read(data.get_model_paths()[catagory][id_].get_obj_path())



base_mesh = createBaseMesh(obj_mesh)
obj_mesh.rotate_vector(vector=(0, 1, 0), angle=rotation, inplace=True)
# Camera Setup
dist_limit = -0.5
camera = pv.Camera()
camera.position = (4.0, 1.0, 4.0)

near_range = 0.3
far_range = 0.8
camera.clipping_range = (near_range, far_range)

frustum = camera.view_frustum(1)

# Calculates the mesh intersection of obj_mesh
position = camera.position
end = (camera.position[0]*dist_limit, camera.position[1]*dist_limit,camera.position[2]*dist_limit)
line = pv.Line(pointa=position, pointb=end, resolution= 1)
points1, ind1 = obj_mesh.ray_trace(position, end)

in1, out1 = outRayFinder(points1)

pl = pv.Plotter(window_size=[windowSize, windowSize]) 

# Adds frustrum
_ = pl.add_mesh(frustum, style="wireframe",color="blue")
# Adds Object Meshes
try:
    obj = pl.add_mesh(obj_mesh, show_edges=show_edges, texture=obj_texture)
except :
    print("Textures Not Working!")
    obj = pl.add_mesh(obj_mesh, show_edges=show_edges)
_ = pl.add_mesh(line, color='red')
_ = pl.add_mesh(base_mesh,color='green')

if in1 is not None:
    intersect11 = pv.Plane(center = in1, direction = (1,0,0))
    intersect12 = pv.Plane(center = out1, direction = (1,0,0))

    if show_planes:
        _ = pl.add_mesh(intersect11, color='r', opacity=0.5)
        _ = pl.add_mesh(intersect12, color='r', opacity=0.5)

    if show_pts:
        _ = pl.add_points(in1, color='blue', point_size=50)
        _ = pl.add_points(out1, color='blue', point_size= 50)

pl.camera = camera
# pl.show_axes()
print("Red Thickness:",  dist(in1, out1) if (not in1 is None) else 0, "PTS = ", in1, out1)
# pl.show(jupyter_backend=backend)
# pl.show()
pl.ren_win.SetOffScreenRendering(5)
pl.screenshot("try.png",transparent_background=True, window_size=[image_res,int(image_res*pl.window_size[1]/pl.window_size[0])])


# # https://github.com/pyvista/pyvista-support/issues/103 High Quality PDF example that helps